# Connected Components

The purpose of this assignment is to familiarize yourself with the handling of graph data structures. You will implement depth-first search for identifying the connected components of an undirected graph, implementing procedure Search as a subroutine along the way.

You will use the [NetworkX](https://networkx.github.io/) Python package to represent and manipulate graphs. You should first familiarize yourself with its functionality by going through the brief [tutorial](http://networkx.github.io/documentation/networkx-1.9.1/tutorial/index.html). For this homework, you may only use the basic undirected graph methods listed [here](http://networkx.github.io/documentation/networkx-1.9.1/reference/classes.graph.html).

As a use case, we will work with a dataset recording the interactions between characters in Homer's *Iliad*.

In [116]:
import networkx
import urllib2
import itertools
homer = urllib2.urlopen('http://people.sc.fsu.edu/~jburkardt/datasets/sgb/homer.dat')

The format of the data is straightforward. After some comment lines (beginning with \*), the file lists a codename for each character (i.e., node of the graph), followed by a description. The file then lists the groups of characters that interact in each chapter, from which you will form the edges. For instance, the first line has the form:

```1:CH,AG,ME,GS;AP,CH;HE,AC;AC,AG,CA;HE,AT;AT,AC;AT,OG;NE,AG,AC;CS,OD```

This means that CH,AG,ME,GS interacted, so there are edges for all pairs of these nodes. Groups of characters that interacted are separated by semicolons. The lines start with chapter information of the form `1:` or `&:`, which can be ignored for this problem.

First implement a function to read in the nodes from the input file. You may implement any auxiliary functions as needed, and are encouraged to use small functions with specific purposes to keep your code readable. Any function you implement should be clearly commented.

In [117]:
def read_nodes(gfile):
    """Reads in the nodes of the graph from the input file.
    
    Args:
        gfile: A handle for the file containing the graph data, starting at the top.
        
    Returns:
        A generator of the nodes in the graph, yielding a list of the form:
            ['CH', 'AG, 'ME', ...]
    """
    # TODO: implement function
    nodeList = []
    """ for each line in  file  , that does not start with * ; Split the line using 'space' 
    and read the  first element into the list . Come out of the loop if the first element in 
    the line is a 'next line char'  """
    for line in gfile:
        temp = line.split(' ')[0]     
        if temp == "\n":
            break
        if temp != "*":
            nodeList.append(temp)
    return nodeList

Next implement a function to read in the edges from the input file.

In [118]:
def read_edges(gfile):
    """Reads in the edges of the graph from the input file.
    
    Args:
        gfile: A handle for the file containing the graph data, starting at the top 
            of the edges section.
            
    Returns:
        A generator of the edges in the graph, yielding a list of pairs of the form:
            [('CH', 'AG'), ('AG', 'ME'), ...]
    """
    # TODO: implement function
    #gfile = urllib2.urlopen('http://people.sc.fsu.edu/~jburkardt/datasets/sgb/homer.dat')
    edgeList = []
   
    for line in gfile:
        
        line = line.rstrip()  # remove new line character from the end of the line
        if ':'  in line:
            temp = line.split(':')
            temp = temp[1]
            temp1 = temp.split(';') 
            for str2 in temp1:
                temp2 = str2.split(',')
                temp3 =list (itertools.combinations(temp2, 2)) # create cobination of two nodes each 
                edgeList.extend(temp3)
    return edgeList

The following code should now correctly create the graph.

In [119]:
import networkx as nx
G = nx.Graph()
G.add_nodes_from(read_nodes(homer))
G.add_edges_from(read_edges(homer))

Next implement procedure Search. The function takes in a graph and a root node, and returns a list of the nodes visited during the search. The nodes should appear in the order in which they were *first visited*. The neighbors of a node should be processed in *alphabetical order*, where numbers come before letters. This will ensure that the output of your function is uniquely defined, given any input node.

In [120]:
def Search(graph, root):
    """Runs depth-first search through a graph, starting at a given root. Neighboring
    nodes are processed in alphabetical order.
    
    Args:
        graph: the given graph, with nodes encoded as strings.
        root: the node from which to start the search.
        
    Returns:
        A list of nodes in the order in which they were first visited.
    """
    # TODO: implement function
    discovered = []
#    output = []
    tmp_neighbour_stk = [root,]
    while tmp_neighbour_stk:
        node = tmp_neighbour_stk.pop()
        if node not in discovered:
            """Add the nodes in reverse sorted order  i.e the lowest index in the list 
            should have the node that we should visit the last . The pop() function will access 
            the  highest index of the list first (which is the first neighbor node that
            we should visit)"""
            node_neighbour = sorted(list(set(graph.neighbors(node))), reverse = True)
            tmp_neighbour_stk.extend(node_neighbour)
            discovered.append(node)
            
           
    return discovered 

We will check the correctness of your code by verifying that it correctly computes the DFS tree starting at Ulysses (node `OD`).

In [121]:
ulysses = Search(G, 'OD')

Next implement DFS to find the connected components of the character graph. When choosing roots for your components, always pick the *smallest unvisited node* according to alphabetical ordering. Combined with your Search routine, this will ensure that the output is again uniquely defined.

In [122]:
def connected_components(graph):
    """Computes the connected components of the given graph.
    
    Args: 
        graph: the given graph, with nodes encoded as strings.
        
    Returns:
        The connected components of the graph. Components are listed in
        alphabetical order of their root nodes.
    """
    # TODO: implement function
    visited = []
    allNodes = graph.nodes()
    output = []
    print len(allNodes)
#    print  set(visited)
    
    while(len(allNodes) > len(visited)):
       node = min(set(allNodes) - set(visited) )
       visited.extend(list(Search(graph,node)))
       output.append(list(Search(graph,node)))
    return output 

We will check correctness of your code by verifying that your output list is identical to our solution.

In [123]:
character_interactions = connected_components(G)

561


As a preliminary check, you should find that the following statements are all true.

In [124]:
component_sizes = [len(c) for c in character_interactions]
print "There are 12 connected components in the Iliad:", len(component_sizes) == 12
print "The giant component has size 542:", max(component_sizes) == 542
print "There are 5 isolated characters:", len([c for c in component_sizes if c == 1]) == 5

There are 12 connected components in the Iliad: True
The giant component has size 542: True
There are 5 isolated characters: True
